<a href="https://colab.research.google.com/github/Himm11/NeuralNetwork-Lab/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LAB 6,7,8 : SENTIMENT ANALYSIS
Submitted by : Himadri Singh (500082972)

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from nltk.corpus import stopwords
from collections import Counter
import string
import re
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

from nltk import WordNetLemmatizer

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
f = '/content/Twitter_Data.csv'
df = pd.read_csv(f)
df=df.iloc[0:7000]
df.head()

#-1 = negative
#0 = neutral
#1 = positive

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [ ]:
X = df['clean_text'].values
X=X.astype(str)
y=df['category'].values


print('shape of train data is' )
print(X.shape)
print('shape of test data is')
print (y.shape)

shape of train data is
(7000,)
shape of test data is
(7000,)


In [ ]:
##Text Pre-processing
#removing punctuation, changing text to lowercase, tokenization
from string import punctuation

def preprocess(x_train):
    all_tweets=list()
    for t in x_train:
        t= t.lower()
        t = "".join([i for i in t if i not in punctuation])
        all_tweets.append(t)
    all_text = " ".join(all_tweets)
    all_words = all_text.split()
    return all_tweets, all_words


all_tweets, all_words = preprocess(X)
print(all_words)

#Lemmatization
wordnet_lemmatizer=WordNetLemmatizer()
def lemmatizer(all_words):
  lemm_text=[wordnet_lemmatizer.lemmatize(word) for word in all_words]
  print(lemm_text)

lemmatizer(all_words)


['when', 'modi', 'promised', '“minimum', 'government', 'maximum', 'governance”', 'expected', 'him', 'begin', 'the', 'difficult', 'job', 'reforming', 'the', 'state', 'why', 'does', 'take', 'years', 'get', 'justice', 'state', 'should', 'and', 'not', 'business', 'and', 'should', 'exit', 'psus', 'and', 'temples', 'talk', 'all', 'the', 'nonsense', 'and', 'continue', 'all', 'the', 'drama', 'will', 'vote', 'for', 'modi', 'what', 'did', 'just', 'say', 'vote', 'for', 'modi', 'welcome', 'bjp', 'told', 'you', 'rahul', 'the', 'main', 'campaigner', 'for', 'modi', 'think', 'modi', 'should', 'just', 'relax', 'asking', 'his', 'supporters', 'prefix', 'chowkidar', 'their', 'names', 'modi', 'did', 'great', 'service', 'now', 'there', 'confusion', 'what', 'read', 'what', 'not', 'now', 'crustal', 'clear', 'what', 'will', 'crass', 'filthy', 'nonsensical', 'see', 'how', 'most', 'abuses', 'are', 'coming', 'from', 'chowkidars', 'answer', 'who', 'among', 'these', 'the', 'most', 'powerful', 'world', 'leader', 'to

In [ ]:
#Counting words
from collections import Counter
count_words = Counter(all_words)
total_words=len(all_words)
sorted_words=count_words.most_common(total_words)

#creating a dictionary to map words to integer value
vocab_to_int={w:i+1 for i,(w,c) in enumerate(sorted_words)}
#start index from 1 rather than 0 coz we might consider it a criteria for padding

print(vocab_to_int)

{'modi': 1, 'the': 2, 'and': 3, 'for': 4, 'you': 5, 'will': 6, 'this': 7, 'not': 8, 'india': 9, 'are': 10, 'that': 11, 'has': 12, 'with': 13, 'have': 14, 'but': 15, 'his': 16, 'from': 17, 'all': 18, 'bjp': 19, 'people': 20, 'congress': 21, 'like': 22, 'they': 23, 'vote': 24, 'who': 25, 'only': 26, 'was': 27, 'what': 28, 'can': 29, 'your': 30, 'now': 31, 'narendra': 32, 'why': 33, 'govt': 34, 'our': 35, 'rahul': 36, 'one': 37, 'money': 38, 'its': 39, 'about': 40, 'dont': 41, 'how': 42, 'modis': 43, 'just': 44, 'country': 45, 'years': 46, 'their': 47, 'him': 48, 'should': 49, 'after': 50, 'more': 51, 'when': 52, 'poor': 53, 'any': 54, 'election': 55, 'indian': 56, 'time': 57, 'than': 58, 'give': 59, 'which': 60, 'then': 61, 'even': 62, 'again': 63, 'via': 64, 'also': 65, 'get': 66, 'want': 67, 'did': 68, 'chowkidar': 69, 'power': 70, 'gandhi': 71, 'out': 72, 'every': 73, 'government': 74, 'there': 75, 'where': 76, 'party': 77, 'against': 78, 'make': 79, 'been': 80, 'them': 81, 'sir': 82,

In [ ]:
#Index based coding
encoded_tweets=list()
for tweet in all_tweets:
  encoded_tweet=list()
  for t in tweet.split():
    if t not in vocab_to_int.keys():
      #if word is not available in vocab_to_int put 0 in that place
      encoded_tweet.append(0)
    else:
      encoded_tweet.append(vocab_to_int[t])
  encoded_tweets.append(encoded_tweet)

print(encoded_tweets)

[[52, 1, 141, 7157, 74, 1294, 7158, 1044, 48, 3863, 2, 1295, 232, 7159, 2, 214, 33, 190, 121, 46, 66, 781, 214, 49, 3, 8, 397, 3, 49, 3864, 3865, 3, 2429], [439, 18, 2, 997, 3, 690, 18, 2, 1386, 6, 24, 4, 1], [28, 68, 44, 134, 24, 4, 1, 1212, 19, 414, 5, 36, 2, 716, 1954, 4, 1, 102, 1, 49, 44, 2430], [317, 16, 504, 2774, 69, 47, 749, 1, 68, 150, 3220, 31, 75, 3221, 28, 202, 28, 8, 31, 7160, 520, 28, 6, 7161, 2178, 7162, 96, 42, 174, 1955, 10, 318, 17, 543], [299, 25, 691, 84, 2, 174, 1213, 163, 126, 167, 874, 3866, 1, 156], [1214, 3222, 7163, 7164, 1045, 1504], [2431, 300, 2179, 2775, 2180, 2432, 1787, 4, 32, 1, 127, 63], [7, 223, 17, 1046, 60, 12, 7165, 22, 1, 1630, 3, 3867, 57, 3868], [13, 1215, 55, 9, 4943, 146, 415, 3869, 311, 1047, 1, 3870, 34, 556, 13, 505, 4944, 7166, 7, 4945, 817, 4946, 2776, 7167], [71, 27, 2433, 190, 1], [355, 22, 945, 576, 7168, 3, 1788, 7169, 2181, 4947, 85, 2777, 1138, 1956, 4948, 134, 11, 98, 59, 7, 51, 57, 135, 4947, 22, 2182, 2, 234, 112, 51, 78, 105, 7

In [ ]:
tweets_len=[len(x) for x in encoded_tweets]

#Removing all tweets of length 0
encoded_tweets= [encoded_tweets[i] for i,l in enumerate(tweets_len) if l>0]
y=[y[i] for i, l in enumerate(tweets_len) if l>0]

#Truncating tweets with maximum length being
sequence_length=70
features=np.zeros((len(encoded_tweets), sequence_length), dtype=int)
#We want to create a 2D metrix where row is representing all tweets
#columns is for the sequence length we have specified(i.e. 70)


for i, tweet in enumerate(encoded_tweets):
  tweet_len=len(tweet)
  if (tweet_len<=sequence_length):
    zeros=list(np.zeros(sequence_length-tweet_len))
    new=zeros+tweet
  else:
    new=tweet[:sequence_length]

features[i,:]=np.array(new)
print(features)

[[    0     0     0 ...     0     0     0]
 [    0     0     0 ...     0     0     0]
 [    0     0     0 ...     0     0     0]
 ...
 [    0     0     0 ...     0     0     0]
 [    0     0     0 ...     0     0     0]
 [    0     0     0 ...  4061    81 16354]]


In [ ]:
#Splitting training and testing dataset
x_train=features[:int(0.7*len(features))]
y_train=y[:int(0.7*len(features))]
x_test=features[int(0.7*len(features)):]
y_test=y[int(0.7*len(features)):]

#x_train=x_train.astype(int)
#x_test=x_test.astype(int)
x_train=torch.tensor(x_train)
y_train=torch.tensor(y_train)
x_test=torch.tensor(x_test)
y_test=torch.tensor(y_test)
print(len(x_train))
print(len(x_test))
print(len(y_train))
print(len(y_test))

4900
2100
4900
2100


In [ ]:
#Creating tensors; Dataloading
import torch
from torch.utils.data import DataLoader, TensorDataset

#create Tensor Dataset
train_data=TensorDataset(torch.tensor(x_train), torch.tensor(y_train))
test_data=TensorDataset(torch.tensor(x_test), torch.tensor(y_test))

#dataloader
batch_size=40
train_loader=DataLoader(train_data, batch_size=batch_size, shuffle=True,drop_last=True)
test_loader=DataLoader(test_data, batch_size=batch_size, shuffle=True,drop_last=True)


<ipython-input-92-4276d507b660>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_data=TensorDataset(torch.tensor(x_train), torch.tensor(y_train))
<ipython-input-92-4276d507b660>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_data=TensorDataset(torch.tensor(x_test), torch.tensor(y_test))


In [ ]:
import torch.nn as nn

class SentimentalLSTM(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        super().__init__()
        self.output_size=output_size
        self.n_layers=n_layers
        self.hidden_dim=hidden_dim

        #Embedding
        self.embedding=nn.Embedding(vocab_size, embedding_dim)
        #LSTM
        self.lstm=nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)

        #dropout layer
        self.dropout=nn.Dropout(0.2)

        #Linear
        self.fc1=nn.Linear(hidden_dim, 112)
        self.fc2=nn.Linear(112, 25)
        self.fc3=nn.Linear(25,output_size)
        #Using Sigmoid function
        self.sig=nn.Sigmoid()

    def forward(self, x, hidden):
        batch_size=x.size()

        #Embedding
        embedd=self.embedding(x)
        #LSTM output
        lstm_out, hidden=self.lstm(embedd, hidden)

        #stack up the lstm output
        lstm_out=lstm_out.contiguous().view(-1, self.hidden_dim)

        #dropout and fully connected layers
        out=self.dropout(lstm_out)
        out=self.fc1(out)
        out=self.dropout(out)
        out=self.fc2(out)
        out=self.dropout(out)
        out=self.fc3(out)
        sig_out=self.sig(out)

        sig_out=sig_out.view(batch_size, -1)
        sig_out=sig_out[:, -1]

        return sig_out, hidden

    def init_hidden(self, batch_size):
        #Initiailise hidden layer
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())

        return hidden

In [ ]:
#Allocating value
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding
output_size = 1
embedding_dim = 200
hidden_dim = 256
n_layers = 2

net = SentimentalLSTM(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
print(net) # loss and optimization functions



SentimentalLSTM(
  (embedding): Embedding(16355, 200)
  (lstm): LSTM(200, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=256, out_features=112, bias=True)
  (fc2): Linear(in_features=112, out_features=25, bias=True)
  (fc3): Linear(in_features=25, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [ ]:
lr=0.001

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

# training params

epochs = 3

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()

for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs=inputs.cuda()
            labels=labels.cuda()
        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        inputs = inputs.type(torch.LongTensor)
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()



    net.train()
    print("Epoch: {}/{}...".format(e+1, epochs),
                  "Loss: {:.6f}...".format(loss.item()))
    # loss is a number indicating how bad the model's prediction was on a single example; the lesser the number , the better the model

Epoch: 1/3... Loss: 44.662460...
Epoch: 2/3... Loss: 44.252171...
Epoch: 3/3... Loss: 18.500149...
